# Ferret DWI tractography

In [1]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


## Imports & Plotting function 

In [2]:
from __future__ import division, print_function, absolute_import

import numpy as np
import nibabel as nib
from dipy.reconst.peaks import peaks_from_model, PeaksAndMetrics
from dipy.core.sphere import Sphere
from dipy.data import get_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.tracking import utils
from common import load_nifti, save_trk, save_peaks, load_peaks, save_trk_old_style
from dipy.viz import actor, window, fvtk

#from ipdb import set_trace

def show_results(streamlines, vol, affine, world_coords=True, opacity=0.6):

    from dipy.viz import actor, window, widget

    shape = data.shape

    if not world_coords:
        from dipy.tracking.streamline import transform_streamlines
        streamlines = transform_streamlines(streamlines, np.linalg.inv(affine))

    ren = window.Renderer()
    if streamlines is not None:
        stream_actor = actor.line(streamlines)

    if not world_coords:
        image_actor = actor.slicer(vol, affine=np.eye(4))
    else:
        image_actor = actor.slicer(vol, affine)

    slicer_opacity = opacity #.6
    image_actor.opacity(slicer_opacity)

    if streamlines is not None:
        ren.add(stream_actor)
    ren.add(image_actor)

    show_m = window.ShowManager(ren, size=(1200, 900))
    show_m.initialize()

    def change_slice(obj, event):
        z = int(np.round(obj.get_value()))
        image_actor.display_extent(0, shape[0] - 1,
                                   0, shape[1] - 1, z, z)

    slider = widget.slider(show_m.iren, show_m.ren,
                           callback=change_slice,
                           min_value=0,
                           max_value=shape[2] - 1,
                           value=shape[2] / 2,
                           label="Move slice",
                           right_normalized_pos=(.98, 0.6),
                           size=(120, 0), label_format="%0.lf",
                           color=(1., 1., 1.),
                           selected_color=(0.86, 0.33, 1.))

    global size
    size = ren.GetSize()

    def win_callback(obj, event):
        global size
        if size != obj.GetSize():

            slider.place(ren)
            size = obj.GetSize()

    show_m.initialize()

    show_m.add_window_callback(win_callback)
    show_m.render()
    show_m.start()

    # ren.zoom(1.5)
    # ren.reset_clipping_range()

    # window.record(ren, out_path='bundles_and_a_slice.png', size=(1200, 900),
    #               reset_camera=False)

    del show_m



### Simple viewers

In [3]:
def simple_viewer(streamlines, vol, affine):

    renderer = window.Renderer()
    renderer.add(actor.line(streamlines))
    renderer.add(actor.slicer(vol, affine))
    window.show(renderer)

In [4]:
def show_gradients(gtab):
    
    renderer = window.Renderer()
    renderer.add(fvtk.point(gtab.gradients, (1,0,0), point_radius=100))
    renderer.add(fvtk.point(-gtab.gradients, (1,0,0), point_radius=100))
    
    window.show(renderer)

## Tracking set of functions

The following set of code allows to do the tracktography of Diffusion Weighted Images. 

The inputs necessary are: 
* a nifti file of the raw data
* its associated bvals and bvecs
* (optional) a mask in nifti

Output:
* FA, evecs & rgb nifti files
* .trk file containing the streamlines
* (annexes) image (png) of the tensor using elipsoids 

Parameters:
* own mask or masking based on the DWI data (median_otsu)
* type of model used. Here TensorModel (DTI). Can also be ConstrainedSphericalDeconvModel (CSD) or others
* type of fiting of the model. Here (by default) Weighted Least Square (WLS). Can be Ordinary Least Square (OLS) or others
* seed density. To increase the number of streamlines
* threshold of the streamline length filter 

### Input file paths and loading

In [5]:
dname = '/Volumes/Samsung_T1/dti/Dipy/P64_F01/'#'/Volumes/Samsung_T1/dti/Dipy/P2_F25/'

fdwi = dname + '2dseq.src.gz.nii.gz'#'2dseq_conv.nii'#'2dseq.src.gz.nii.gz' #nii from 2dseq in dsi_studio

data, affine = load_nifti(fdwi)

#fbval = dname + 'P64_F01.bval' #'bvals' #from dsi_studio
#fbvec = dname + 'P64_F01.bvec' #'bvecs'

#bvals, bvecs = read_bvals_bvecs(fbval, fbvec)

#gtab = gradient_table(bvals, bvecs, b0_threshold=50)

#fmask = dname + 'mask_extern.nii' #None if wish to get an automated mask from dwi data

In [6]:
#dsiStudio
print(data.shape)
print(data.max())
print(data.min())

(120, 160, 80, 201)
32408
0


In [7]:
dname2 = '/Volumes/Samsung_T1/dti/ExploreDTI/P64_F01/'#'/Volumes/Samsung_T1/dti/Dipy/P2_F25/'

fdwi2 = dname2 + '2dseq_conv.nii'#'2dseq.src.gz.nii.gz' #nii from 2dseq in dsi_studio

data2, affine2 = load_nifti(fdwi2)

#ExploreDTI
print(data2.shape)
print(data2.max())
print(data2.min())

(160, 120, 80, 210)
32766
0


In [47]:
fbval2 = dname2 + 'P64_F01.bval' #'bvals' #from dsi_studio
fbvec2 = dname2 + 'P64_F01.bvec' #'bvecs'

bvals2, bvecs2 = read_bvals_bvecs(fbval2, fbvec2)

gtab2 = gradient_table(bvals2, bvecs2, b0_threshold=10)

In [51]:
gtab2.bvecs.shape

(210, 3)

In [42]:
bvals2.shape

(210,)

In [9]:
data[60:70,80:90,:,100].shape

(10, 10, 80)

In [18]:
data[60,80,40,100]

1969

In [19]:
data2[60,80,40,100]

16720

In [55]:
data_test = data2/32000

In [34]:
import matplotlib.pyplot as plt

#fig_hist, ax = plt.subplots(1)
plt.figure(1)
plt.pcolor(data[:,:,40,100])#, color='burlywood')
#ax.set_xlabel('Values')
#ax.set_ylabel('Count')
plt.colorbar()
plt.show()
#plt.legend()
#plt.savefig('length_histogram.png')

In [35]:
#fig_hist, ax = plt.subplots(1)
plt.figure(2)
plt.pcolor(data2[:,:,40,100])#, color='burlywood')
#ax.set_xlabel('Values')
#ax.set_ylabel('Count')
plt.colorbar()
plt.show()
#plt.legend()
#plt.savefig('length_histogram.png')

In [27]:
#checks for gtab
#show_gradients(gtab)
np.sum(gtab.b0s_mask)
print(gtab.info)
plot(np.sort(gtab.bvals))

B-values shape (210,)
         min 0.000000 
         max 4117.299451 
B-vectors shape (210, 3)
         min -0.996774 
         max 1.000000 
None


### Hand-made mask or autogenerated mask from the DWI data

In [7]:
if fmask is None: 
    from dipy.segment.mask import median_otsu
    b0_mask, mask = median_otsu(data)  # TODO: check parameters to improve the mask
else: 
    mask, mask_affine = load_nifti(fmask)
    mask = np.squeeze(mask) #fix mask dimensions 

### DTI Model computation and fitting (further for CSD--not tested)

In [52]:
# compute DTI model
from dipy.reconst.dti import TensorModel
tenmodel = TensorModel(gtab2)#, fit_method='OLS') #, min_signal=5000)

In [56]:
# fit the dti model
tenfit = tenmodel.fit(data_test)#, mask=mask)

### DWI indicators computation and saving in nifti files (FA, first eigen vector, rgb tensor)

In [33]:
from dipy.reconst.dti import fractional_anisotropy, color_fa, lower_triangular

FA = fractional_anisotropy(tenfit.evals)

fa_img = nib.Nifti1Image(FA.astype(np.float32), affine)
nib.save(fa_img, dname+'tensor_fa-2.nii.gz')

In [57]:
# save fa
ffa = dname2 + 'tensor_fa_2.nii.gz'

fa_img = nib.Nifti1Image(tenfit.fa.astype(np.float32), affine)
nib.save(fa_img, ffa)

In [106]:
# save first eigen vector
evecs_img = nib.Nifti1Image(tenfit.evecs.astype(np.float32), affine)
nib.save(evecs_img, dname+'tensor_evecs.nii.gz')

In [107]:
# compute and save rgb
from dipy.reconst.dti import color_fa
RGB = color_fa(tenfit.fa, tenfit.evecs)
nib.save(nib.Nifti1Image(np.array(255 * RGB, 'uint8'), affine), dname+'tensor_rgb.nii.gz')

In [11]:
sh_order = 8 #TODO: check what that does
if data.shape[-1] < 15:
    raise ValueError('You need at least 15 unique DWI volumes to '
                     'compute fiber ODFs. You currently have: {0}'
                     ' DWI volumes.'.format(data.shape[-1]))
elif data.shape[-1] < 30:
    sh_order = 6

In [12]:
# compute the response equation ?
from dipy.reconst.csdeconv import auto_response

response, ratio = auto_response(gtab, data)

response = list(response)

In [13]:
peaks_sphere = get_sphere('symmetric362')

#for CSD

from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
model = ConstrainedSphericalDeconvModel(gtab, response,
                                        sh_order=sh_order)

In [14]:
#TODO: check what that does
peaks_csd = peaks_from_model(model=model,
                             data=data,
                             sphere=peaks_sphere,
                             relative_peak_threshold=.5, #.5
                             min_separation_angle=25, #25
                             mask=mask,
                             return_sh=True,
                             sh_order=sh_order,
                             normalize_peaks=True,
                             parallel=False)

In [15]:
peaks_csd.affine = affine

fpeaks = dname + 'peaks.npz'

save_peaks(fpeaks, peaks_csd)

In [16]:
from dipy.io.trackvis import save_trk
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier,
                                 LocalTracking)

stopping_thr = 0.1   #0.25

pam = load_peaks(fpeaks)

In [17]:
#ffa = dname + 'tensor_fa_nomask.nii.gz'
fa, fa_affine = load_nifti(ffa)

classifier = ThresholdTissueClassifier(fa,
                                       stopping_thr)

In [18]:
# seeds 

seed_density = 1

seed_mask = fa > 0.1 #0.4 #TODO: check this parameter

seeds = utils.seeds_from_mask(
    seed_mask,
    density=seed_density,
    affine=affine)

In [19]:
#TODO: check what that does

#if use_sh:
#    detmax_dg = \
#        DeterministicMaximumDirectionGetter.from_shcoeff(
#            pam.shm_coeff,
#            max_angle=30.,
#            sphere=pam.sphere)
#
#    streamlines = \
#        LocalTracking(detmax_dg, classifier, seeds, affine,
#                      step_size=.5)
#
#else:


# tractography, if affine then in world coordinates
streamlines = LocalTracking(pam, classifier,
                            seeds, affine=affine, step_size=.12) #.5

# Compute streamlines and store as a list.
streamlines = list(streamlines)

ftractogram = dname + 'tractogram_nomask.trk'

#save .trk
save_trk_old_style(ftractogram, streamlines, affine, fa.shape)

#render
#show_results(streamlines, fa, fa_affine)

In [93]:
# threshold on streamline length

from dipy.tracking.utils import length
lengths = list(length(streamlines))

print(max(length(streamlines)))
print(min(length(streamlines)))

45.36
0.48


In [98]:
new_streamlines = [ s for s, l in zip(streamlines, lengths) if l > 1. ] #3.5

new_streamlines = list(new_streamlines)
new_lengths = list(length(new_streamlines))

In [99]:
# info length streamlines

print(len(streamlines))
print(len(new_streamlines))

print(max(length(streamlines)))
print(min(length(streamlines)))

print(max(length(new_streamlines)))
print(min(length(new_streamlines)))

534916
502022
45.36
0.48
45.36
1.2


In [100]:
# show new tracto

fnew_tractogram = dname + 'filteredtractogram.trk'
#save_trk_old_style(fnew_tractogram, new_streamlines, affine, fa.shape)

show_results(new_streamlines, fa, fa_affine, opacity=0.6)

In [94]:
import matplotlib.pyplot as plt

fig_hist, ax = plt.subplots(1)
ax.hist(lengths, color='burlywood')
ax.set_xlabel('Length')
ax.set_ylabel('Count')
plt.show()
plt.legend()
#plt.savefig('length_histogram.png')


In [28]:
import matplotlib.pyplot as plt

fig_hist, ax = plt.subplots(1)
ax.hist(new_lengths, color='burlywood')
ax.set_xlabel('Length')
ax.set_ylabel('Count')
plt.show()
plt.legend()
#plt.savefig('length_histogram.png')

## Annexes

In [84]:
data.shape

(120, 160, 80, 201)

In [44]:
print('Computing tensor ellipsoids in a part of the splenium of the CC')

from dipy.data import get_sphere
sphere = get_sphere('symmetric724')

from dipy.viz import fvtk
ren = fvtk.ren()

evals = tenfit.evals[40:80,40:120,39:40] #[13:43, 44:74, 28:29]
evecs = tenfit.evecs[40:80,40:120,39:40]

Computing tensor ellipsoids in a part of the splenium of the CC


In [45]:
cfa = RGB[40:80,40:120,39:40]
cfa /= cfa.max()

fvtk.add(ren, fvtk.tensor(evals, evecs, cfa, sphere))

print('Saving illustration as tensor_ellipsoids.png')
fvtk.record(ren, n_frames=1, out_path=dname+'tensor_ellipsoids.png', size=(1000, 1000))

/Users/ghfc/anaconda/lib/python2.7/site-packages/dipy/viz/fvtk.py:998: RuntimeWarning: invalid value encountered in true_divide
  ea /= ea.max()


Saving illustration as tensor_ellipsoids.png


In [102]:
fvtk.record(ren, n_frames=1, out_path=dname+'tensor_ellipsoids.png', size=(1000, 1500))

In [ ]:
# first endpoint of the first streamline
streamlines[0][0]

In [ ]:
# other endpoint of the first streamline
streamlines[0][-1]